<a href="https://colab.research.google.com/github/bhuvanikavijay/shadowfox-/blob/main/Advanced_level_shadowfox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv('/content/amazon_alexa.tsv', sep='\t')
data.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [ ]:
mydata = data[['verified_reviews','feedback']]
mydata.columns = ['review','label']

mydata.head()

,review,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
mydata.value_counts('label')

,count
label,
1,2893
0,257


In [ ]:
label_counts = mydata["label"].value_counts()

# Get the number of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

# Drop rows from the majority class randomly
if rows_to_drop > 0:
   data_majority = mydata[mydata["label"] == 1]
   data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
   data_balanced = mydata.copy()

# Check the new class balance
print(data_balanced["label"].value_counts())

label
1    257
0    257
Name: count, dtype: int64


In [ ]:
import re

def clean_text(text):
  # Remove special characters and punctuation
  text = re.sub(r"[^\w\s]", " ", text)

  # Remove single characters
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)

  # Remove HTML tags
  text = re.sub(r"<[^>]*>", " ", text)

  # Lowercase the text
  text = text.lower()

  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text)

  # Trim leading and trailing spaces
  text = text.strip()

  return text

In [ ]:
import re
import pandas as pd

def clean_text(text):
  # Check if the input is a string
  if isinstance(text, str):
    # Remove special characters and punctuation
    text = re.sub(r"[^\w\s]", " ", text)

    # Remove single characters
    text = re.sub(r"\b[a-zA-Z]\b", " ", text)

    # Remove HTML tags
    text = re.sub(r"<[^>]*>", " ", text)

    # Lowercase the text
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text)

    # Trim leading and trailing spaces
    text = text.strip()

    return text
  else:
    # Handle non-string values (e.g., return an empty string or a placeholder)
    return ""

# Extract the review column as a list
reviews = data_balanced['review'].tolist()

# Clean the text in the list
cleaned_reviews = [clean_text(review) for review in reviews]

# Add the cleaned reviews as a new column to the DataFrame
data_balanced['clean_reviews'] = cleaned_reviews

In [ ]:
data_balanced

,review,label,clean_reviews
4,Music,1,music
19,I liked the original Echo. This is the same bu...,1,liked the original echo this is the same but s...
35,I bought this to compare the speaker quality t...,1,bought this to compare the speaker quality to ...
46,"It's like Siri, in fact, Siri answers more acc...",0,it like siri in fact siri answers more accurat...
62,Did t really know what I could use this device...,1,did really know what could use this device for...
...,...,...,...
3067,The only negative we have on this product is t...,0,the only negative we have on this product is t...
3091,I didn’t order it,0,didn order it
3096,The product sounded the same as the emoji spea...,0,the product sounded the same as the emoji spea...
3108,"I live alone, this is the best companion!",1,live alone this is the best companion


In [ ]:
import pandas as pd

# Assuming your DataFrame is called "df"
total_rows = len(data_balanced)
test_size = int(total_rows * 0.95)

# Randomly sample train_size rows for the training set
test_set = data_balanced.sample(test_size)

# Get the remaining rows for the test set
train_set = data_balanced.drop(test_set.index)

In [ ]:
test_set.shape

(488, 3)

In [ ]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,review,label,clean_reviews,pred_label
3048,NOT CONNECTED TO MY PHONE PLAYLIST :(,0,not connected to my phone playlist,
2581,This device does not interact with my home fil...,0,this device does not interact with my home fil...,
1393,Love my spot! My favorite feature is the &#34;...,1,love my spot my favorite feature is the 34 ale...,
1487,Very disappointed. Alexa doesn’t hear me and ...,0,very disappointed alexa doesn hear me and does...,
1895,I’m still unable to connect any devices althou...,0,still unable to connect any devices although i...,
...,...,...,...,...
2008,I only purchase the echo plus because it has a...,0,only purchase the echo plus because it has bui...,
491,I've already returned it.,0,ve already returned it,
2613,Ask it to play Motown radio on Pandora and it ...,0,ask it to play motown radio on pandora and it ...,
1704,Great addition to our Alexa family. Great vide...,1,great addition to our alexa family great video...,


In [ ]:
batches = []
batch_size = 50

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])

In [ ]:
!pip install openai
import openai
import time

def gpt_completion_function(batch,current_batch,total_batch,train_sample,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  sample_json_data = train_sample[['clean_reviews','label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.

  ```
  {json_data}
  ```
  ####
  {sample_json_data}
  ####
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openai
import openai
import time

def gpt_completion_function(batch,current_batch,total_batch,train_sample,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  sample_json_data = train_sample[['clean_reviews','label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.
   {sample_json_data}
  ####
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  # Use openai.chat.completions.create instead of openai.ChatCompletion.create
  response = openai.chat.completions.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]



In [ ]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Clean the data by stripping the backticks
  json_data = response.strip("`")

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  df_temp = pd.DataFrame(data)

  # Append the DataFrame to the final DataFrame
  df_total = df_total.append(df_temp, ignore_index=True)

print(df_total)  # Display the final DataFrame

Empty DataFrame
Columns: []
Index: []


In [ ]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Clean the data by stripping the backticks
  json_data = response.strip("`")
  print(f"json_data: {json_data}") # Print to check the content

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  print(f"data: {data}") # Print to check the content

  df_temp = pd.DataFrame(data)
  print(f"df_temp: {df_temp}") # Print to check the content

  # Check if 'pred_label' exists in df_temp
  if 'pred_label' in df_temp.columns:
    # Append the DataFrame to the final DataFrame (consider using concat for better performance)
    df_total = pd.concat([df_total, df_temp], ignore_index=True)
  else:
    print(f"'pred_label' not found in df_temp for response: {response}")

print(df_total)  # Display the final DataFrame

Empty DataFrame
Columns: []
Index: []


In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 16.4 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [ ]:
import google.generativeai as genai

genai.configure(api_key='YOUR_API_KEY') # Set your API Key here

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
import google.generativeai as genai

# Make sure to replace 'YOUR_API_KEY' with your actual API key
genai.configure(api_key='YOUR_ACTUAL_API_KEY')

In [41]:
!pip install -q -U google-generativeai

In [42]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [51]:
test_set_sample = test_set.sample(20)

test_set_sample['pred_label'] = ''

test_set_sample

,review,label,clean_reviews,pred_label
813,Upgraded from my lilttle echo dot. Happy with ...,1,upgraded from my lilttle echo dot happy with s...,
2659,Sound is terrible. Cannot pair with echo to pl...,0,sound is terrible cannot pair with echo to pla...,
1416,The fact that she can help me find my phone wh...,1,the fact that she can help me find my phone wh...,
1291,"item returned for repair ,receivded item back ...",0,item returned for repair receivded item back f...,
1945,It was harder to setup then I thought it shoul...,0,it was harder to setup then thought it should ...,
1463,Love it!! Still learning all the possibilities.,1,love it still learning all the possibilities,
2159,Love it! It's everything I wanted and more,1,love it it everything wanted and more,
1577,,1,,
2049,"I love Alexa, she works great but she not for ...",1,love alexa she works great but she not for lit...,
1386,Invasive and scared the crap out of me for spe...,0,invasive and scared the crap out of me for spe...,


from matplotlib import pyplot as plt
test_set_sample['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test_set_sample['label'].plot(kind='line', figsize=(8, 4), title='label')
plt.gca().spines[['top', 'right']].set_visible(False)

In [52]:
json_data = test_set_sample[['clean_reviews','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"clean_reviews":"upgraded from my lilttle echo dot happy with sound","pred_label":""},{"clean_reviews":"sound is terrible cannot pair with echo to play simultaneously must upgrade to amazon music if you want to play more than one device at time","pred_label":""},{"clean_reviews":"the fact that she can help me find my phone when am at home alone is worth the price of this device","pred_label":""},{"clean_reviews":"item returned for repair receivded item back from repair 07 23 18 parts missing no power cord included please advise","pred_label":""},{"clean_reviews":"it was harder to setup then thought it should be for some reason had to default it twice had to go thru too many steps to setup and find my devices would be nice if it could interact with my xfinity cable tv cable box and it also does not work with my xfinity alarm for my house or my cameras be nice set and disarm the alarm with alexa had to install skill app to setup bulb whats that about you ship it with your product shoul

In [53]:
prompt = f"""
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
{json_data}
```
"""

print(prompt)


You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
[{"clean_reviews":"upgraded from my lilttle echo dot happy with sound","pred_label":""},{"clean_reviews":"sound is terrible cannot pair with echo to play simultaneously must upgrade to amazon music if you want to play more than one device at time","pred_label":""},{"clean_reviews":"the fact that she can help me find my phone when am at home alone is worth the price of this device","pred_label":""},{"clean_reviews":"item returned for repair receivded item back from repair 07 23 18 parts missing no power cord

In [56]:
!pip install google-colab

import google.colab.userdata

# Store the API key.
# Example conditional
if 'my_api_key' in locals():
    google.colab.userdata.set('my_api_key', 'your_api_key')

In [61]:
!pip install openai==0.27.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.47.1
    Uninstalling openai-1.47.1:
      Successfully uninstalled openai-1.47.1


In [63]:
test_set.shape

(488, 3)

In [64]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,review,label,clean_reviews,pred_label
1503,"Alexa is pretty dumb ,,,",0,alexa is pretty dumb,
62,Did t really know what I could use this device...,1,did really know what could use this device for...,
382,Poor quality. Gave it away.,0,poor quality gave it away,
896,I owned an echo for overa year but the new lac...,1,owned an echo for overa year but the new lacks...,
1375,"Given as a gift to my wife, to compliment my e...",1,given as gift to my wife to compliment my echo...,
...,...,...,...,...
928,I am not super impressed with Alexa. When my P...,0,am not super impressed with alexa when my prim...,
2005,Why do we need to buy a $100 hub to get it to ...,0,why do we need to buy 100 hub to get it to wor...,
2305,Had to return it as Hulu does not work on devi...,0,had to return it as hulu does not work on devi...,
2593,Alexa is phenomenal!,1,alexa is phenomenal,


from matplotlib import pyplot as plt
_df_0['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['label'].plot(kind='line', figsize=(8, 4), title='label')
plt.gca().spines[['top', 'right']].set_visible(False)

In [65]:
batches = []
batch_size = 50

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])

In [67]:
import time

def gpt_completion_function(batch,current_batch,total_batch,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  {json_data}
  ```
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

In [74]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Clean the data by stripping the backticks
  json_data = response.strip("`")

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  df_temp = pd.DataFrame(data)

  # Append the DataFrame to the final DataFrame
  df_total = df_total.append(df_temp, ignore_index=True)

print(df_total)  # Display the final DataFrame

Empty DataFrame
Columns: []
Index: []


In [ ]:
import time
import json
import pandas as pd
!pip install openai

# Assuming you have `openai` api key set in your environment.
# If not, you can set it using the command below:
# !export OPENAI_API_KEY='YOUR_API_KEY'

def gpt_completion_function(batch,current_batch,total_batch,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  # The following two lines were incorrectly indented
  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.

  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

In [80]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,review,label,clean_reviews,pred_label
2136,We love the Fire TV Stick and will love it eve...,1,we love the fire tv stick and will love it eve...,
550,Absolutely love it!,1,absolutely love it,
2541,Cheap and cheap sound.,0,cheap and cheap sound,
1240,I haven't figured out how to make or receive c...,0,haven figured out how to make or receive calls...,
341,Alexa hardly came on..,0,alexa hardly came on,
...,...,...,...,...
1918,I am so disappointed in this product. I tried...,0,am so disappointed in this product tried to in...,
673,Device was not working. Followed all instructi...,0,device was not working followed all instructio...,
376,Doesn't always respond when spoken to with pro...,0,doesn always respond when spoken to with promp...,
338,Beyond my capabilities to program. Grandkids l...,1,beyond my capabilities to program grandkids lv...,


from matplotlib import pyplot as plt
test_set_total['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test_set_total['label'].plot(kind='line', figsize=(8, 4), title='label')
plt.gca().spines[['top', 'right']].set_visible(False)

In [81]:
batches = []
batch_size = 50

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])

In [82]:
import time

def gpt_completion_function(batch,current_batch,total_batch,train_sample,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  sample_json_data = train_sample[['clean_reviews','label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.

  ```
  {json_data}
  ```
  ####
  {sample_json_data}
  ####
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

In [ ]:
import time
import json
import pandas as pd
!pip install openai

# Assuming you have `openai` api key set in your environment.
# If not, you can set it using the command below:
# !export OPENAI_API_KEY='YOUR_API_KEY'

def gpt_completion_function(batch,current_batch,total_batch,train_sample,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  sample_json_data = train_sample[['clean_reviews','label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.

In [85]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Clean the data by stripping the backticks
  json_data = response.strip("`")

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  df_temp = pd.DataFrame(data)

  # Append the DataFrame to the final DataFrame
  df_total = df_total.append(df_temp, ignore_index=True)

print(df_total)

Empty DataFrame
Columns: []
Index: []


In [89]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle

In [92]:
import sys
sys.path.append('/path/to/directory/containing/nlp_utils')

In [93]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [95]:
dataset = pd.read_csv('/content/Restaurant_Reviews.tsv',sep = '\t')

In [96]:
dataset

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


from matplotlib import pyplot as plt
dataset['Liked'].plot(kind='hist', bins=20, title='Liked')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dataset['Liked'].plot(kind='line', figsize=(8, 4), title='Liked')
plt.gca().spines[['top', 'right']].set_visible(False)

In [97]:
stopset = set(stopwords.words('english'))

In [98]:
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stopset)

In [99]:
dataset.Review

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
995,I think food should have flavor and texture an...
996,Appetite instantly gone.
997,Overall I was not impressed and would not go b...
998,"The whole experience was underwhelming, and I ..."


In [100]:
vectorizer

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents='ascii')

In [101]:
stoplist = list(stopset)

# Now create the vectorizer with the stoplist
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stoplist)

#The rest of your code should work as expected.
X = vectorizer.fit_transform(dataset.Review)
y = dataset.Liked

In [102]:
pickle.dump(vectorizer, open('tranform.pkl', 'wb'))

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [104]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB()

In [105]:
pred = clf.predict(X_test)

In [106]:
accuracy_score(y_test,pred)*100

77.5

In [107]:
clf = naive_bayes.MultinomialNB()
clf.fit(X,y)

MultinomialNB()

In [108]:
accuracy_score(y_test,clf.predict(X_test))*100

94.0

In [109]:
filename = 'nlp_model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [110]:
from sklearn.ensemble import RandomForestClassifier

In [111]:
model = RandomForestClassifier(n_estimators = 501,
                            criterion = 'entropy')

model.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=501)

In [112]:
y_pred = model.predict(X_test)

y_pred


array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1])

In [113]:
accuracy_score(y_test,y_pred)*100

71.0